In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vi-vqa-edu/Verify_Convert_80.csv


In [2]:
import logging
import os
import re
import sys
import unicodedata

import gdown
import pandas as pd
from datasets import Dataset
from PIL import Image

content = "kaggle/working"
os.chdir("/" + content + "/")
print(os.getcwd())
DELETE_MYFOLDER = True


def delete_directory_contents(path):
    """Safely delete all files and subfolders in the given path"""
    if not os.path.exists(path):
        print(f"Path does not exist: {path}")
        return False

    try:
        # List all files and directories
        for item in os.listdir(path):
            item_path = os.path.join(path, item)

            # If it's a file, delete it
            if os.path.isfile(item_path):
                os.unlink(item_path)
            # If it's a directory, delete it and all its contents
            elif os.path.isdir(item_path):
                import shutil
                shutil.rmtree(item_path)

        print(f"Successfully deleted all contents in: {path}")
        return True
    except Exception as e:
        print(f"Error deleting contents: {e}")
        return False


if DELETE_MYFOLDER:
    delete_directory_contents(f"/{content}/MyFolder")
os.makedirs(f"/{content}/MyFolder", exist_ok=True)
os.chdir(f"/{content}/MyFolder")
print(os.getcwd())

!nvidia-smi

/kaggle/working
Path does not exist: /kaggle/working/MyFolder
/kaggle/working/MyFolder
Thu Jun 19 17:55:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |          

In [3]:
import os
import zipfile

import gdown

# Google Drive file ID and output file name
file_id = "147uMVzvFkmzm__njqPHH1oodtgYOcqdm"
output = "downloaded_file.zip"

# Construct the direct download URL
url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(url, output, quiet=False)

# Unzip the file
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("unzipped_folder")  # Extract to 'unzipped_folder'

print("Download and extraction complete.")

Downloading...
From (original): https://drive.google.com/uc?id=147uMVzvFkmzm__njqPHH1oodtgYOcqdm
From (redirected): https://drive.google.com/uc?id=147uMVzvFkmzm__njqPHH1oodtgYOcqdm&confirm=t&uuid=6146a5d6-1a96-4df4-9e5c-ffa69d4584dc
To: /kaggle/working/MyFolder/downloaded_file.zip
100%|██████████| 1.42G/1.42G [00:18<00:00, 79.0MB/s]


Download and extraction complete.


# Runs 

In [4]:
!pip install transformers datasets evaluate bitsandbytes peft trl pillow accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstall

In [5]:
import random


def normalize_vqa_vietnamese(text):
    # Unicode normalization (NFC is standard for Vietnamese)
    text = unicodedata.normalize('NFC', text)
    # Lowercase
    text = text.lower()
    # Remove punctuation (keep Vietnamese letters and numbers)
    # text = re.sub(
    #     r'[^\w\sàáảãạăằắẳẵặâầấẩẫậèéẻẽẹêềếểễệìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵđ]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# Configure logging to print to console
# Custom format: [TIME] LEVEL: message

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True,  # NOTE: Must force
    stream=sys.stdout  # Send logs to stdout
)
logging.info("Logging here")


def load_data_from_csv(csv_file, image_folder, cut_size=100):
    df = pd.read_csv(csv_file)
    # Limit the number of rows to cut_size for faster testing
    df = df.head(cut_size).reset_index(drop=True)

    # Pre-validate images to avoid runtime errors
    valid_data = []
    from tqdm import tqdm
    for _, row in tqdm(df.iterrows()):
        image_id = row['ImageID']
        category = image_id.split('_')[0]
        image_path = os.path.join(image_folder, category, f"{image_id}.png")

        if os.path.exists(image_path):
            try:
                # Pre-load and validate image
                img = Image.open(image_path).convert('RGB')
                # Resize to consistent size for faster processing
                img = img.resize((224, 224), Image.Resampling.LANCZOS)
                # Append valid data
                valid_data.append({
                    'image': img,
                    'question': normalize_vqa_vietnamese(row['Question']),
                    'answer': normalize_vqa_vietnamese(row['Answer'])
                })
            except Exception as e:
                logging.error(f"Error processing image {image_path}: {e}")
        else:
            logging.warning(f"Image not found: {image_path}")

    return Dataset.from_list(valid_data)


NUM = 1000 * 18 
csv_file = '/kaggle/input/vi-vqa-edu/Verify_Convert_80.csv'
image_folder = '/kaggle/working/MyFolder/unzipped_folder/ViVQA4Edu'
train_ds = load_data_from_csv(csv_file, image_folder, NUM)

dev_size = int(0.05 * len(train_ds))
dev_ds = train_ds.shuffle(42).select(range(dev_size))
print(train_ds, train_ds[0], dev_ds, end="\n\n")

[2025-06-19 17:57:28] INFO: Logging here


18000it [02:39, 113.04it/s]


Dataset({
    features: ['image', 'question', 'answer'],
    num_rows: 18000
}) {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=224x224 at 0x7971977C2CD0>, 'question': 'các bạn học sinh đang làm gì trong lớp học?', 'answer': 'các bạn học sinh đang thực hiện các hoạt động khác nhau trong lớp học như lau dọn bàn ghế, quét sàn và cầm sách vở, cho thấy lớp học đang được chuẩn bị cho buổi học hoặc một hoạt động nào đó.'} Dataset({
    features: ['image', 'question', 'answer'],
    num_rows: 900
})



In [ ]:
import torch
from huggingface_hub import login
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    BitsAndBytesConfig,
    PaliGemmaForConditionalGeneration,
    PaliGemmaProcessor,
    Trainer,
    TrainingArguments,
)

# Prompt for your Hugging Face token
login(token="Your API Key")


bnb_config = BitsAndBytesConfig(
    # load_in_4bit=True,
    # bnb_4bit_compute_type=torch.bfloat16, # NOTE: This is quan type 
    # bnb_4bit_quant_type="nf4",
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",  # Match your input dtype
)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj", "v_proj",
                    "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model_id = 'google/paligemma-3b-pt-224'
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},  # Adjust device_map as needed
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("Model dtype:", model.dtype)

2025-06-19 18:04:18.530141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750356258.743530      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750356258.810577      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850
Model dtype: torch.float16


In [ ]:
import torch
from huggingface_hub import login
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    BitsAndBytesConfig,
    PaliGemmaForConditionalGeneration,
    PaliGemmaProcessor,
    Trainer,
    TrainingArguments,
)

# Prompt for your Hugging Face token
login(token="Your API Key")


# model_id = 'google/paligemma-3b-pt-224'
# model 
processor = PaliGemmaProcessor.from_pretrained(model_id)
print(processor.tokenizer.convert_tokens_to_ids('<image>')) 
print(processor.tokenizer.convert_tokens_to_ids('trả lời câu hỏi bằng tiếng việt <vi> <image>')) 

def collate_fn(examples):
    # examples is a list of dictionaries with keys: "question", "answer", "image"
    prompts = ["trả lời câu hỏi bằng tiếng việt: <image>" + example["question"] for example in examples]
    labels = [example["answer"] for example in examples]
    images = [example["image"] for example in examples]
    prc = processor(
        text=prompts, images=images, suffix=labels,
        return_tensors="pt", padding="longest", 
        # tokenize_newline_separately=False # Not valid 
    )
    prc = prc.to(model.dtype)
    return prc 
    

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

257152
3


In [8]:
tmp = collate_fn([train_ds[0], train_ds[1]])

for k, v in tmp.items():
    if isinstance(v, torch.Tensor):
        print(f"{k}: shape={v.shape}, dtype={v.dtype}, range=[{v.min()}, {v.max()}]")
    else:
        print(f"{k}: type={type(v)}")

input_ids: shape=torch.Size([2, 329]), dtype=torch.int64, range=[0, 257152]
token_type_ids: shape=torch.Size([2, 329]), dtype=torch.int64, range=[0, 1]
attention_mask: shape=torch.Size([2, 329]), dtype=torch.int64, range=[0, 1]
pixel_values: shape=torch.Size([2, 3, 224, 224]), dtype=torch.float16, range=[-1.0, 1.0]
labels: shape=torch.Size([2, 329]), dtype=torch.int64, range=[-100, 236177]


In [9]:
import logging
import time
from datetime import datetime

from transformers import (
    BitsAndBytesConfig,
    EarlyStoppingCallback,
    PaliGemmaForConditionalGeneration,
    PaliGemmaProcessor,
    Trainer,
    TrainerCallback,
    TrainingArguments,
)


class TimedStoppingCallback(TrainerCallback):
    """
    A callback that stops training after a specified duration.
    """

    def __init__(self, max_duration_seconds: int, check_steps: int):
        self.max_duration_seconds = max_duration_seconds
        self.start_time = None
        self.check_steps = check_steps  # Check time every N steps

    def on_train_begin(self, args, state, control, **kwargs):
        """Called at the beginning of training."""
        self.start_time = time.time()
        logging.info(
            f"Training started. Will stop after approximately {self.max_duration_seconds / 3600:.2f} hours.")

    def on_step_end(self, args, state, control, **kwargs):
        """Called at the end of each training step."""
        if state.global_step % self.check_steps == 0 and self.start_time is not None:
            elapsed_time = time.time() - self.start_time
            if elapsed_time > self.max_duration_seconds:
                logging.info(
                    f"Stopping training after {elapsed_time:.2f} seconds (~{elapsed_time / 3600:.2f} hours) due to time limit.")
                control.should_training_stop = True


def get_args(repo_name: str, train_ds, time_training: int):
    repo_name = repo_name.replace("\\", "_")
    repo_name = repo_name.replace("/", "_")
    str_now = datetime.now().strftime("%d%H%M")
    repo_name = f"{repo_name.strip()}_{str_now}"
    logging.info(f"Output directory: {repo_name}")
    # Define training arguments
    global_bs = 16
    save_steps = min(50, len(train_ds) // global_bs)  # Save every n steps
    save_steps = max(save_steps, 2)  # Ensure at least 2 steps
    check_steps = save_steps // 2  # Check every half of save steps
    patience = 10  # Early stopping patience in turns

    # Custom callback to stop training after a certain time
    callbacks = [
        TimedStoppingCallback(
            max_duration_seconds=time_training, check_steps=check_steps),
        # Use compare from load best model for early stopping
        EarlyStoppingCallback(
            early_stopping_patience=patience,  # Early stopping patience
            early_stopping_threshold=0.01,  # Threshold for early stopping
        )
    ]

    training_args = {
        "seed": 42,  # Random seed for reproducibility
        "per_device_train_batch_size": 2, # NOTE: Maximum 
        "per_device_eval_batch_size": 2,
        # Adjusted for global batch size
        "gradient_accumulation_steps": 8,
        "bf16": True,
        "gradient_checkpointing": False,  # Disable gradient checkpointing
        "dataloader_pin_memory": False,  # Disable pin memory for compatibility
        "eval_strategy": "steps",
        "report_to": ["tensorboard"],
        "push_to_hub": True,
        "save_total_limit": 1,
        "load_best_model_at_end": True,
        "metric_for_best_model": "eval_loss", # NOTE: Use eval loss 
        "greater_is_better": False,
        "num_train_epochs": 5,
        "warmup_steps": 2,
        "learning_rate": 2e-5,
        "weight_decay": 1e-6,
        "adam_beta2": 0.999,
        "optim": "adamw_torch",
        "max_grad_norm": 1.0,
    }

    args = TrainingArguments(
        remove_unused_columns=False,
        output_dir=repo_name,
        save_steps=save_steps,
        eval_steps=save_steps,
        logging_steps=save_steps,
        **training_args,
    )
    return args, callbacks


def train_model(model, repo_name, train_ds, eval_ds, time_training):
    args, callbacks = get_args(repo_name, train_ds, time_training)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        data_collator=collate_fn,
        # label_names=["labels"],  # Specify the label field(s) here
        callbacks=callbacks
    )
    logging.info("***** Running training *****")
    trainer.train()
    logging.info("***** Training completed *****")
    trainer.push_to_hub(commit_message="Training complete")

    return trainer


In [10]:
repo_name = "pali"
time_training = 3600 * 9 
trainer = train_model(model, repo_name, train_ds, dev_ds, time_training)

[2025-06-19 18:05:49] INFO: Output directory: pali_191805


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


[2025-06-19 18:05:50] INFO: ***** Running training *****
[2025-06-19 18:05:50] INFO: Training started. Will stop after approximately 9.00 hours.


Step,Training Loss,Validation Loss
50,18.213300,1.776556
100,11.969400,1.304270
150,9.762500,1.194029
200,9.057600,1.132498
250,9.328600,1.090589
300,8.543500,1.058610
350,8.250800,1.035714
400,8.364200,1.015050
450,8.034300,0.998173
500,8.153700,0.981815


[2025-06-20 03:03:40] INFO: ***** Training completed *****
